In [1]:
from pyspark.ml.feature import RegexTokenizer, HashingTF, IDF, CountVectorizer, Normalizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, DoubleType, StringType, StructType, StructField
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.ml import Pipeline, PipelineModel
import pyspark

# Data Retrieval

In [2]:
data_file = r"/home/jovyan/repos/distributed-sentiment-analysis-on-twitter-data/twitter_data/*.csv"
vader_analyzer = SentimentIntensityAnalyzer()

In [3]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '6g'),
                                   ('spark.driver.memory','6g'),
                                   ('spark.master', 'local[*]')])

# Initialize a Spark session
spark = SparkSession \
    .builder \
    .appName("SentimentAnalysis") \
    .config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext

In [4]:
sc.getConf().getAll()

[('spark.executor.memory', '6g'),
 ('spark.driver.host', 'de4f1c03e850'),
 ('spark.driver.memory', '6g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '35327'),
 ('spark.app.name', 'SentimentAnalysis'),
 ('spark.app.id', 'local-1524117988418'),
 ('spark.ui.showConsoleProgress', 'true')]

In [5]:
# define the data schema(format/structure) for our twitter data in the csv file
twitter_data_schema = StructType([StructField("date_str", StringType(), True),
                                  StructField("tweet_id", StringType(), True),
                                  StructField("text", StringType(), True),
                                  StructField("location", StringType(), True),
                                  StructField("user_id", StringType(), True),
                                  StructField("user_name", StringType(), True),
                                  StructField("user_location", StringType(), True),
                                  StructField("user_url", StringType(), True),
                                  StructField("user_description", StringType(), True),
                                  StructField("place_id", StringType(), True),
                                  StructField("place_url", StringType(), True),
                                  StructField("place_type", StringType(), True),
                                  StructField("place_countrycode", StringType(), True),
                                  StructField("place_country", StringType(), True),
                                  StructField("place_boundingboxtype", StringType(), True),
                                  StructField("entities_hashtags", StringType(), True),
                                  StructField("entities_urls", StringType(), True),
                                  StructField("entities_mentions", StringType(), True),
                                  StructField("entities_symbols", StringType(), True),
                                  StructField("entities_media", StringType(), True),
                                  StructField("entities_polls", StringType(), True),])

In [6]:
df_raw = spark.read.csv(
    data_file, schema=twitter_data_schema
)

In [7]:
text_and_loc_only = df_raw.select("text", "location")

In [8]:
text_and_loc_only.cache()

text_and_loc_only.count()

1593023

# Tweet Cleaning Function

In [9]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

at_user_pat = r'@[A-Za-z0-9_]+'  # r'@[\w]+'
url_pat = r'https?://[^ ]+'  # r'https?:\/\/[^\s]+'
www_pat = r'www.[^ ]+'
repeating_chars_pat = r'([A-Za-z])\1+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(at_user_pat, 'USERNAME', bom_removed)
    stripped = re.sub(url_pat, 'URL', stripped)
    stripped = re.sub(www_pat, 'URL', stripped)
    stripped = re.sub(repeating_chars_pat, r'\1\1', stripped)

    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

# Data Pre-Processing

In [10]:
df_filtered = text_and_loc_only.filter("location like '%,___'")
df_filtered.show(truncate=True)

+--------------------+----------------+
|                text|        location|
+--------------------+----------------+
|Walking to work t...|  Cincinnati, OH|
|Can you recommend...|      Medley, FL|
|Interested in a #...|     Warwick, RI|
|Desiigner an indu...|        Mesa, AZ|
|Thank you Jesus:f...|  Greensboro, NC|
|How do I deactiva...|       Xenia, OH|
|She a pussy hoe t...|Philadelphia, PA|
|Want to work at S...|      Mystic, CT|
|   USER_NAME Heemy76|Philadelphia, PA|
|I think I wanna m...|     Memphis, TN|
|If you're looking...|    Stamford, CT|
|We're #hiring! Cl...|       Sayre, PA|
|Can you recommend...|     Commack, NY|
|This #job might b...|     Augusta, ME|
|Interested in a #...|  Huntington, IN|
|Interested in a #...|     Massena, NY|
|We're #hiring! Cl...|      Laurel, MD|
|We're #hiring! Cl...|     Norwalk, CT|
|Want to work in #...|     Kittery, ME|
|We're looking for...|    Newburgh, NY|
+--------------------+----------------+
only showing top 20 rows



In [11]:
df_filtered.count()

960362

In [12]:
udf_tweet_cleaner = udf(tweet_cleaner)
df_preprocessed = df_filtered.withColumn("text", udf_tweet_cleaner(col("text")))
df_preprocessed.show(truncate=True)

+--------------------+----------------+
|                text|        location|
+--------------------+----------------+
|walking to work t...|  Cincinnati, OH|
|can you recommend...|      Medley, FL|
|interested in job...|     Warwick, RI|
|desiigner an indu...|        Mesa, AZ|
|thank you jesus f...|  Greensboro, NC|
|how do deactivate...|       Xenia, OH|
|she pussy hoe til...|Philadelphia, PA|
|want to work at s...|      Mystic, CT|
|     user name heemy|Philadelphia, PA|
|think wanna miss ...|     Memphis, TN|
|if you re looking...|    Stamford, CT|
|we re hiring clic...|       Sayre, PA|
|can you recommend...|     Commack, NY|
|this job might be...|     Augusta, ME|
|interested in job...|  Huntington, IN|
|interested in job...|     Massena, NY|
|we re hiring clic...|      Laurel, MD|
|we re hiring clic...|     Norwalk, CT|
|want to work in k...|     Kittery, ME|
|we re looking for...|    Newburgh, NY|
+--------------------+----------------+
only showing top 20 rows



In [13]:
df_preprocessed.cache()

df_preprocessed.count()

960362

In [14]:
def count(text):
    return 1

udf_count = udf(count)

In [45]:
MIN_TWEETS_COUNT = 1000

# Ranking Result of NLTK Analyzer 

In [15]:
def nltk_prediction(text):
    result = vader_analyzer.polarity_scores(str(text))
    prediction = 1 if result['compound'] >= 0 else 0

    return prediction

udf_nltk_prediction = udf(nltk_prediction)

In [16]:
def nltk_score(text):
    result = vader_analyzer.polarity_scores(str(text))
    prediction = result['compound']

    return prediction

udf_nltk_score = udf(nltk_score)

In [17]:
df_scored = df_preprocessed.withColumn("prediction", udf_nltk_prediction(col("text")))
df_scored = df_scored.withColumn("score", udf_nltk_score(col("text")))
df_scored = df_scored.withColumn("count", udf_count(col("text")))

df_scored.cache()
df_scored.count()

960362

In [42]:
# Ranking by averaging 1/0 values
df_scored.groupBy('location').agg({'prediction': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(prediction)', ascending=False).show(n=1000)

+--------------------+------------------+----------+
|            location|   avg(prediction)|sum(count)|
+--------------------+------------------+----------+
|      Pittsburgh, PA|0.8332270408163265|    3136.0|
|  Salt Lake City, UT|0.8275154004106776|    1461.0|
|        St Louis, MO|0.8236130867709816|    1406.0|
|         Wichita, KS|0.8215994531784006|    1463.0|
|           Plano, TX|0.8211446740858506|    1258.0|
|       Las Vegas, NV|0.8193859121011439|    4983.0|
|      Scottsdale, AZ|0.8137583892617449|    1192.0|
|   San Francisco, CA|0.8115262649937957|    7253.0|
|           Miami, FL|0.8093645484949833|    3588.0|
|       Manhattan, NY|0.8088080168776371|   22752.0|
|         Orlando, FL|0.8067542213883677|    2665.0|
|       Arlington, VA|0.8024464831804281|    1635.0|
|     Minneapolis, MN|0.8023743016759777|    2864.0|
|   Winston-Salem, NC|0.8020989505247377|    1334.0|
|          Queens, NY|0.8016814292148327|    6661.0|
|         Raleigh, NC| 0.800465580312604|    3

In [43]:
# Ranking by averaging compound scores
df_scored.groupBy('location').agg({'score': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(score)', ascending=False).show(n=1000)

+--------------------+-------------------+----------+
|            location|         avg(score)|sum(count)|
+--------------------+-------------------+----------+
|        St Louis, MO|0.18028819345661445|    1406.0|
|           Plano, TX| 0.1754244833068362|    1258.0|
|           Omaha, NE|0.17393708269131994|    1947.0|
|         Wichita, KS| 0.1726965140123034|    1463.0|
|           Tulsa, OK|0.17210634674922595|    1292.0|
|       Arlington, VA|0.17073314984709465|    1635.0|
|     Kansas City, MO|0.16382691090004523|    2211.0|
|  Salt Lake City, UT|0.16254647501711153|    1461.0|
|      Scottsdale, AZ|0.15786854026845634|    1192.0|
|         Raleigh, NC|0.15748895909544383|    3007.0|
|   Oklahoma City, OK|0.15568032623169098|    3004.0|
|          Austin, TX|0.15333244652406422|    7480.0|
|         El Paso, TX|0.15294541125541117|    2310.0|
|Colorado Springs, CO|0.15270353356890456|    1415.0|
|          Boston, MA|0.14928714206539803|    5413.0|
|        Paradise, NV|0.1489

# Ranking Result of Logistic Regression

In [20]:
def extract_probability(input):
    return input.toArray().tolist()[1]

udf_extract_probability = udf(extract_probability)

In [21]:
trigramwocs_pipelineFit_loaded = PipelineModel.load("ngram_cv_idf_lr")

In [22]:
%%time
df_ngram_cv_idf_lr = trigramwocs_pipelineFit_loaded.transform(df_preprocessed)

df_ngram_cv_idf_lr = df_ngram_cv_idf_lr.withColumn("count", udf_count(col("text")))
df_ngram_cv_idf_lr = df_ngram_cv_idf_lr.withColumn("score", udf_extract_probability(df_ngram_cv_idf_lr.probability))

df_ngram_cv_idf_lr.cache()
df_ngram_cv_idf_lr.count()

CPU times: user 80 ms, sys: 30 ms, total: 110 ms
Wall time: 1min 21s


In [46]:
# Ranking by averaging 1/0 values
df_ngram_cv_idf_lr.groupBy('location').agg({'prediction': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(prediction)', ascending=False).show(n=1000)

+--------------------+------------------+----------+
|            location|   avg(prediction)|sum(count)|
+--------------------+------------------+----------+
|      Scottsdale, AZ|0.7843959731543624|    1192.0|
|       Las Vegas, NV|0.7822596829219346|    4983.0|
|       Manhattan, NY|0.7821290436005626|   22752.0|
|         St Paul, MN|0.7764034253092293|    1051.0|
|           Miami, FL|0.7748049052396878|    3588.0|
|       Arlington, VA|0.7712538226299694|    1635.0|
|     Minneapolis, MN|0.7695530726256983|    2864.0|
|        St Louis, MO|0.7681365576102418|    1406.0|
|         Seattle, WA|0.7622656561627443|    5014.0|
|  Salt Lake City, UT|0.7618069815195072|    1461.0|
|          Boston, MA|0.7589137262146683|    5413.0|
|   San Francisco, CA|0.7583069074865573|    7253.0|
|        Brooklyn, NY|0.7552816901408451|   10224.0|
|   St Petersburg, FL|0.7549407114624506|    1012.0|
|         Orlando, FL|0.7527204502814259|    2665.0|
|       San Diego, CA| 0.752412425644415|    7

In [47]:
# Ranking by averaging probabilities
df_ngram_cv_idf_lr.groupBy('location').agg({'score': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(score)', ascending=False).show(n=1000)

+--------------------+------------------+----------+
|            location|        avg(score)|sum(count)|
+--------------------+------------------+----------+
|       Arlington, VA|0.6708742147273796|    1635.0|
|        St Louis, MO|0.6708325668901346|    1406.0|
|       Manhattan, NY|0.6700273485001346|   22752.0|
|      Scottsdale, AZ|0.6639159697349479|    1192.0|
|         Seattle, WA|0.6611632693903476|    5014.0|
|     Minneapolis, MN|0.6611049804879935|    2864.0|
|         St Paul, MN|0.6598993696939583|    1051.0|
|  Salt Lake City, UT| 0.659592531889071|    1461.0|
|           Miami, FL|0.6593047146310471|    3588.0|
|   San Francisco, CA|0.6573471277060481|    7253.0|
|          Boston, MA|0.6563913221576761|    5413.0|
|Colorado Springs, CO|0.6546208717719122|    1415.0|
|   St Petersburg, FL|0.6535607652348434|    1012.0|
|           Omaha, NE|0.6531468635974941|    1947.0|
|     Kansas City, MO|0.6523004051613569|    2211.0|
|       Knoxville, TN|0.6515337010656462|    1

# Ranking Result of Naive Bayes

In [25]:
nb_pipelineFit_loaded = PipelineModel.load("ngram_cv_idf_nb")

In [26]:
%%time
df_ngram_cv_idf_nb = nb_pipelineFit_loaded.transform(df_preprocessed)

df_ngram_cv_idf_nb = df_ngram_cv_idf_nb.withColumn("count", udf_count(col("text")))
df_ngram_cv_idf_nb = df_ngram_cv_idf_nb.withColumn("score", udf_extract_probability(df_ngram_cv_idf_nb.probability))

df_ngram_cv_idf_nb.cache()
df_ngram_cv_idf_nb.count()

CPU times: user 80 ms, sys: 20 ms, total: 100 ms
Wall time: 1min 25s


In [48]:
# Ranking by averaging 1/0 values
df_ngram_cv_idf_nb.groupBy('location').agg({'prediction': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(prediction)', ascending=False).show(n=1000)

+--------------------+------------------+----------+
|            location|   avg(prediction)|sum(count)|
+--------------------+------------------+----------+
|      Scottsdale, AZ| 0.764261744966443|    1192.0|
|           Miami, FL|0.7522296544035675|    3588.0|
|       Manhattan, NY|0.7463959212376934|   22752.0|
|         St Paul, MN|0.7402473834443387|    1051.0|
|         Seattle, WA|0.7389309932189868|    5014.0|
|        St Louis, MO|0.7368421052631579|    1406.0|
|     Minneapolis, MN|0.7328910614525139|    2864.0|
|       Arlington, VA|0.7314984709480122|    1635.0|
|          Boston, MA|0.7308331793829669|    5413.0|
|  Salt Lake City, UT| 0.728952772073922|    1461.0|
|           Omaha, NE|0.7282999486389317|    1947.0|
|   San Francisco, CA|0.7274231352543775|    7253.0|
|   St Petersburg, FL|0.7213438735177866|    1012.0|
|          Denver, CO|0.7204638801642909|    4139.0|
|Colorado Springs, CO|0.7173144876325088|    1415.0|
| Fort Lauderdale, FL|0.7170124481327801|    1

In [49]:
# Ranking by averaging probabilities
df_ngram_cv_idf_nb.groupBy('location').agg({'score': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(score)', ascending=False).show(n=1000)

+--------------------+------------------+----------+
|            location|        avg(score)|sum(count)|
+--------------------+------------------+----------+
|      Scottsdale, AZ|0.7510832915663779|    1192.0|
|           Miami, FL|0.7471619425345617|    3588.0|
|       Manhattan, NY|0.7439800655700508|   22752.0|
|         St Paul, MN|0.7402283554651218|    1051.0|
|         Seattle, WA| 0.735611105053338|    5014.0|
|        St Louis, MO|0.7328592037973607|    1406.0|
|     Minneapolis, MN|0.7310592638919375|    2864.0|
|       Arlington, VA| 0.728068632631937|    1635.0|
|          Boston, MA|0.7270282682956619|    5413.0|
|   San Francisco, CA|0.7252780228424734|    7253.0|
|  Salt Lake City, UT|0.7251330260787349|    1461.0|
|           Omaha, NE|0.7240930859395718|    1947.0|
|       Las Vegas, NV|0.7211747536708224|    4983.0|
|   St Petersburg, FL|0.7173393205213174|    1012.0|
|          Denver, CO|0.7163583544732613|    4139.0|
| Fort Lauderdale, FL|0.7148291479917959|    1

# Ranking Result of Gradient Boost Tree

In [29]:
gbt_pipelineFit_loaded = PipelineModel.load("ngram_cv_idf_gbt")

In [30]:
%%time
df_ngram_cv_idf_gbt = gbt_pipelineFit_loaded.transform(df_preprocessed)

df_ngram_cv_idf_gbt = df_ngram_cv_idf_gbt.withColumn("count", udf_count(col("text")))
df_ngram_cv_idf_gbt = df_ngram_cv_idf_gbt.withColumn("score", udf_extract_probability(df_ngram_cv_idf_gbt.probability))

df_ngram_cv_idf_gbt.cache()
df_ngram_cv_idf_gbt.count()

CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 1min 28s


In [50]:
# Ranking by averaging 1/0 values
df_ngram_cv_idf_gbt.groupBy('location').agg({'prediction': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(prediction)', ascending=False).show(n=1000)

+--------------------+-------------------+----------+
|            location|    avg(prediction)|sum(count)|
+--------------------+-------------------+----------+
|           Miami, FL| 0.5641025641025641|    3588.0|
|       Manhattan, NY| 0.5255362165963432|   22752.0|
|       Arlington, VA| 0.5033639143730887|    1635.0|
|         Orlando, FL| 0.5024390243902439|    2665.0|
|        Paradise, NV| 0.4970913321698662|    3438.0|
|      Washington, DC| 0.4951217288228321|   10967.0|
|         Memphis, TN| 0.4937425506555423|    3356.0|
|     Kansas City, MO|0.49344188150158297|    2211.0|
|      Louisville, KY| 0.4924165824064712|    1978.0|
|   Oklahoma City, OK|0.48501997336884156|    3004.0|
|        St Louis, MO|  0.484352773826458|    1406.0|
|        Richmond, VA| 0.4833916083916084|    2288.0|
|          Newark, NJ|0.48328488372093026|    1376.0|
|      Providence, RI| 0.4829443447037702|    1114.0|
|         Raleigh, NC| 0.4822081809112072|    3007.0|
|   Winston-Salem, NC|0.4812

In [51]:
# Ranking by averaging probabilities
df_ngram_cv_idf_gbt.groupBy('location').agg({'score': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(score)', ascending=False).show(n=1000)

+--------------------+-------------------+----------+
|            location|         avg(score)|sum(count)|
+--------------------+-------------------+----------+
|           Miami, FL| 0.5012493508181272|    3588.0|
|       Knoxville, TN| 0.4976333926016323|    1788.0|
|         Wichita, KS| 0.4958093351229608|    1463.0|
|       Manhattan, NY|0.49526398279755673|   22752.0|
|         Raleigh, NC| 0.4939748036563767|    3007.0|
|         Memphis, TN| 0.4937799938588579|    3356.0|
|           Omaha, NE| 0.4937207780029819|    1947.0|
|        Paradise, NV| 0.4933984089190469|    3438.0|
|         Buffalo, NY| 0.4923418791605366|    1791.0|
|        Richmond, VA| 0.4923020475342615|    2288.0|
|           Plano, TX|0.49227573842741384|    1258.0|
|      Louisville, KY|0.49181196757278706|    1978.0|
|     Minneapolis, MN| 0.4914916397452657|    2864.0|
|     Kansas City, MO|0.49130222148605934|    2211.0|
|       Arlington, VA| 0.4912890865157912|    1635.0|
|         Orlando, FL| 0.491

# Ranking Result of Linear SVM

In [33]:
lsvc_pipelineFit_loaded = PipelineModel.load("ngram_cv_idf_lsvc")

In [34]:
%%time
df_ngram_cv_idf_lsvc = lsvc_pipelineFit_loaded.transform(df_preprocessed)

df_ngram_cv_idf_lsvc = df_ngram_cv_idf_lsvc.withColumn("count", udf_count(col("text")))
df_ngram_cv_idf_lsvc = df_ngram_cv_idf_lsvc.withColumn("score", udf_extract_probability(df_ngram_cv_idf_lsvc.rawPrediction))

df_ngram_cv_idf_lsvc.cache()
df_ngram_cv_idf_lsvc.count()

CPU times: user 80 ms, sys: 10 ms, total: 90 ms
Wall time: 1min 23s


In [52]:
# Ranking by averaging 1/0 values
df_ngram_cv_idf_lsvc.groupBy('location').agg({'prediction': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(prediction)', ascending=False).show(n=1000)

+--------------------+------------------+----------+
|            location|   avg(prediction)|sum(count)|
+--------------------+------------------+----------+
|       Manhattan, NY|0.7835355133614628|   22752.0|
|           Miami, FL|0.7803790412486065|    3588.0|
|      Scottsdale, AZ|0.7776845637583892|    1192.0|
|       Las Vegas, NV|0.7774433072446317|    4983.0|
|         St Paul, MN|0.7754519505233112|    1051.0|
|        St Louis, MO|0.7681365576102418|    1406.0|
|         Seattle, WA|0.7654567211806941|    5014.0|
|  Salt Lake City, UT|0.7624914442162902|    1461.0|
|       Arlington, VA|0.7620795107033639|    1635.0|
|     Minneapolis, MN|0.7618715083798883|    2864.0|
|          Boston, MA|0.7607611306114909|    5413.0|
|   San Francisco, CA|0.7555494278229699|    7253.0|
|   St Petersburg, FL|0.7519762845849802|    1012.0|
|        Brooklyn, NY|0.7517605633802817|   10224.0|
|       San Diego, CA| 0.751487111698612|    7565.0|
|          Queens, NY| 0.750187659510584|    6

In [53]:
# Ranking by averaging raw predictions
df_ngram_cv_idf_lsvc.groupBy('location').agg({'score': 'avg', 'count': 'sum'}).filter(col('sum(count)') > MIN_TWEETS_COUNT).sort('avg(score)', ascending=False).show(n=1000)

+--------------------+-------------------+----------+
|            location|         avg(score)|sum(count)|
+--------------------+-------------------+----------+
|       Arlington, VA| 1.3189516053170116|    1635.0|
|   St Petersburg, FL| 0.7464099254065003|    1012.0|
|       Knoxville, TN| 0.6969658877838982|    1788.0|
|        Paradise, NV| 0.6511372698943234|    3438.0|
|       Manhattan, NY| 0.6132942242113527|   22752.0|
|           Plano, TX| 0.6037554897779112|    1258.0|
|          Denver, CO| 0.5887711873748827|    4139.0|
|        St Louis, MO| 0.5849137914541248|    1406.0|
|         Seattle, WA| 0.5732369029386295|    5014.0|
|Colorado Springs, CO| 0.5712971027927639|    1415.0|
|         Phoenix, AZ| 0.5710574860645671|    6481.0|
|      Scottsdale, AZ| 0.5659615567966046|    1192.0|
|           Miami, FL| 0.5495396988709027|    3588.0|
|          Boston, MA| 0.5462772786045099|    5413.0|
|         Wichita, KS| 0.5447801365735983|    1463.0|
|         St Paul, MN| 0.542